## Model Development

Now that we have our cleaned data with required features, lets proceed with model development 

Importing Libraries

In [1]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import pickle

from xgboost import XGBRegressor
#from lightgbm import LGBMRegressor
#import lightgbm as ltb

from math import sqrt

%matplotlib inline

Loading our data into dataframe

In [2]:
df_properties = pd.read_csv('cleaned_property_data_buy.csv')
#df_properties = df_properties.dropna(column='amenities',axis=1)
df_properties = df_properties.drop(columns = ['amenities','price_per_sqft','neighborhood'],axis=1)

df_properties = pd.get_dummies(df_properties, columns=['quality', ])
df_properties.head()
#partly_furnished

,id,latitude,longitude,price,size_in_sqft,no_of_bedrooms,no_of_bathrooms,partly_furnished,balcony,barbecue_area,...,shared_spa,study,vastu_compliant,view_of_landmark,view_of_water,walk_in_closet,quality_High,quality_Low,quality_Medium,quality_Ultra
0,8942144,25.190442,55.283397,2183160,1040,2,2,True,True,False,...,False,False,False,True,False,False,0,0,1,0
1,8942145,25.190442,55.283397,2190930,1043,2,2,True,True,False,...,False,False,False,True,False,False,0,0,1,0
2,8942143,25.190442,55.283397,2024610,964,2,2,True,True,False,...,False,True,False,True,True,True,1,0,0,0
3,8942138,24.311142,54.621775,1550000,185,3,5,False,True,True,...,False,True,False,False,True,False,1,0,0,0
4,8942134,25.099380,55.141275,6699555,2000,3,3,False,True,True,...,False,False,False,False,True,False,0,0,1,0


In [3]:
#df_properties.price = df_properties.price * 0.27

In [4]:

df_properties.price.describe()

count    2.024000e+03
mean     2.516963e+06
std      4.145156e+06
min      2.150000e+05
25%      9.000000e+05
50%      1.600000e+06
75%      2.794848e+06
max      9.999989e+07
Name: price, dtype: float64

Training dataframe

In [5]:
df_properties.head()

,id,latitude,longitude,price,size_in_sqft,no_of_bedrooms,no_of_bathrooms,partly_furnished,balcony,barbecue_area,...,shared_spa,study,vastu_compliant,view_of_landmark,view_of_water,walk_in_closet,quality_High,quality_Low,quality_Medium,quality_Ultra
0,8942144,25.190442,55.283397,2183160,1040,2,2,True,True,False,...,False,False,False,True,False,False,0,0,1,0
1,8942145,25.190442,55.283397,2190930,1043,2,2,True,True,False,...,False,False,False,True,False,False,0,0,1,0
2,8942143,25.190442,55.283397,2024610,964,2,2,True,True,False,...,False,True,False,True,True,True,1,0,0,0
3,8942138,24.311142,54.621775,1550000,185,3,5,False,True,True,...,False,True,False,False,True,False,1,0,0,0
4,8942134,25.099380,55.141275,6699555,2000,3,3,False,True,True,...,False,False,False,False,True,False,0,0,1,0


Testing dataframe

In [6]:
df_properties.shape

(2024, 35)

Target feature for prediction

In [7]:
y_train = df_properties['price']#.values
#type(y_train)

Excluding target feature from the training dataset

In [8]:
y_train.head()

0    2183160
1    2190930
2    2024610
3    1550000
4    6699555
Name: price, dtype: int64

In [9]:
X = df_properties.drop(columns=['price','id'],axis=1)
#X['maid_room'] = df["maid_room"].astype(int)
#X['maid_room'] = X['maid_room'].replace({True: 1, False: 0})
X = X.replace({False: 0, True: 1}, inplace=False)
#unfurnished
#X['partly_furnished'] = X['partly_furnished'].replace({1: 0, 0: 1})
#X = X[['latitude','longitude','size_in_sqft','no_of_bedrooms','no_of_bathrooms','covered_parking','unfurnished','concierge','kitchen_appliances','pets_allowed','view_of_water']]
y = y_train

In [10]:
X.head()

,latitude,longitude,size_in_sqft,no_of_bedrooms,no_of_bathrooms,partly_furnished,balcony,barbecue_area,built_in_wardrobes,central_ac,...,shared_spa,study,vastu_compliant,view_of_landmark,view_of_water,walk_in_closet,quality_High,quality_Low,quality_Medium,quality_Ultra
0,25.190442,55.283397,1040,2,2,1,1,0,1,1,...,0,0,0,1,0,0,0,0,1,0
1,25.190442,55.283397,1043,2,2,1,1,0,1,1,...,0,0,0,1,0,0,0,0,1,0
2,25.190442,55.283397,964,2,2,1,1,0,1,1,...,0,1,0,1,1,1,1,0,0,0
3,24.311142,54.621775,185,3,5,0,1,1,1,1,...,0,1,0,0,1,0,1,0,0,0
4,25.099380,55.141275,2000,3,3,0,1,1,1,1,...,0,0,0,0,1,0,0,0,1,0


Splitting training dataset for model training.

In [11]:
# Split into train & test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [84]:
X_train.head()

,latitude,longitude,size_in_sqft,no_of_bedrooms,no_of_bathrooms,partly_furnished,balcony,barbecue_area,built_in_wardrobes,central_ac,...,shared_spa,study,vastu_compliant,view_of_landmark,view_of_water,walk_in_closet,quality_High,quality_Low,quality_Medium,quality_Ultra
1196,25.192505,55.288665,708,1,2,1,1,1,1,1,...,0,0,0,1,1,0,1,0,0,0
1820,25.186682,55.282568,905,2,3,0,1,0,1,1,...,0,0,0,0,0,0,0,0,1,0
290,25.096219,55.170272,1566,2,2,0,1,0,1,1,...,0,1,0,0,0,0,0,0,1,0
386,25.038406,55.180451,624,1,2,1,1,0,1,1,...,0,0,0,1,0,1,0,0,1,0
1928,25.112384,55.248247,1382,2,3,0,1,0,1,1,...,0,0,0,0,0,0,0,0,1,0


In [12]:
X.shape

(2024, 33)

In [13]:
X_train.shape

(1619, 33)

For prediction modelling we will use following models:
    
- LGBMRegressor
- RandomForestRegressor
- XGBRegressor
- LGBMRegressor

We will train our models using training dataset consisting of 1619 properties and then test them against our testing dataset with 2024-1619 properties.

We will check our R2 score for each model and we will select the predictions from the model which is closest to value of 1. R2 scores range from 0 to 1.

Based on best R2 score we will match actual values with predicted values and see the percentage difference.

### GradientBoostingRegressor

#### Training

In [14]:
#X_train.head()

In [15]:

#np.random.seed(42)

# for i in [0.001, 0.003, 0.1, 0.3, 1, 2, 4, 6]:
#     for j in [100, 250, 500, 700, 750, 800, 850, 900, 950, 1000, 1050]:
#        gradient = GradientBoostingRegressor(n_estimators = j, learning_rate=i)
#        print('learning rate is: '+ str(i)+ ' and n_estimators is: '+str(j))
#        gradient.fit(X_train, y_train)
#        gradient_predictions = gradient.predict(X_test)
#        gradient_r2_score = r2_score(y_test, gradient_predictions)
#        print('R2 Score for GradientBoostingRegressor', gradient_r2_score)
#        print('\n')
 

#### Testing

In [16]:
#gradient_predictions = gradient.predict(df_test)
gradient = GradientBoostingRegressor(n_estimators = 750)
gradient.fit(X_train, y_train)
       
gradient_predictions = gradient.predict(X_test)
gradient_r2_score = r2_score(y_test, gradient_predictions)
print('R2 Score for GradientBoostingRegressor', gradient_r2_score)

from sklearn.metrics import mean_absolute_error
print("MAE",mean_absolute_error(y_test, gradient_predictions))

filename = 'GradientBoostingRegressor2.sav'
pickle.dump(gradient, open(filename, 'wb'))

R2 Score for GradientBoostingRegressor 0.5986284652208123
MAE 938026.007550355


In [17]:
"""
param_test2 = {'max_depth':range(5,16,2), 'min_samples_split':range(200,1001,200)}
gradient = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.1, n_estimators=60, max_features='sqrt', subsample=0.8, random_state=10), 
param_grid = param_test2, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gradient.fit(train[predictors],train[target])
gradient.grid_scores_, gradient.best_params_, gradient.best_score_
"""

"\nparam_test2 = {'max_depth':range(5,16,2), 'min_samples_split':range(200,1001,200)}\ngradient = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.1, n_estimators=60, max_features='sqrt', subsample=0.8, random_state=10), \nparam_grid = param_test2, scoring='roc_auc',n_jobs=4,iid=False, cv=5)\ngradient.fit(train[predictors],train[target])\ngradient.grid_scores_, gradient.best_params_, gradient.best_score_\n"

### RandomForestRegressor

#### Training

In [18]:

for j in [2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26]:
       random_forest = RandomForestRegressor(max_depth = j)#, learning_rate=i)
       print(' and max_depth is: '+str(j))
       random_forest.fit(X_train, y_train)
       forest_prediction = random_forest.predict(X_test)
       forest_r2_score = r2_score(y_test, forest_prediction)
       print('R2 Score for RandomForestRegressor', forest_r2_score)
       print('\n')
 


 and max_depth is: 2
R2 Score for RandomForestRegressor 0.4720191676790616


 and max_depth is: 4
R2 Score for RandomForestRegressor 0.5975244836228994


 and max_depth is: 6
R2 Score for RandomForestRegressor 0.6480336790448996


 and max_depth is: 8
R2 Score for RandomForestRegressor 0.6774753715735216


 and max_depth is: 10
R2 Score for RandomForestRegressor 0.6641647275868572


 and max_depth is: 12
R2 Score for RandomForestRegressor 0.6513611639707033


 and max_depth is: 14
R2 Score for RandomForestRegressor 0.6726737725463299


 and max_depth is: 16
R2 Score for RandomForestRegressor 0.679769217514532


 and max_depth is: 18
R2 Score for RandomForestRegressor 0.6720257355355799


 and max_depth is: 20
R2 Score for RandomForestRegressor 0.6582388949130971


 and max_depth is: 22
R2 Score for RandomForestRegressor 0.7079194345097497


 and max_depth is: 24
R2 Score for RandomForestRegressor 0.66572007292795


 and max_depth is: 26
R2 Score for RandomForestRegressor 0.696155256955

In [62]:
random_forest = RandomForestRegressor(max_depth=16)
random_forest.fit(X_train, y_train)
#forest_prediction = random_forest.predict(X_test)

RandomForestRegressor(max_depth=16)

#### Testing

In [63]:
forest_prediction = random_forest.predict(X_test)
forest_r2_score = r2_score(y_test, forest_prediction)
print('R2 Score for RandomForestRegressor', forest_r2_score)

from sklearn.metrics import mean_absolute_error
print("MAE",mean_absolute_error(y_test, forest_prediction))

filename = 'random_forest2.sav'
pickle.dump(random_forest, open(filename, 'wb'))

R2 Score for RandomForestRegressor 0.6774610979343318
MAE 957448.9009983557


### XGBRegressor

#### Training

In [21]:
for i in [0.001, 0.003, 0.1, 0.3, 1, 2, 4, 6]:
    for j in [50, 100, 250, 500, 700, 750, 800, 850, 900, 950, 1000, 1050]:
       xgbr_regressor = XGBRegressor(learning_rate=i, n_estimators=j, n_jobs=-1)
       xgbr_regressor.fit(X_train, y_train)
       
       xgbr_regressor = random_forest.predict(X_test)
       xgbreg_r2_score = r2_score(y_test, xgbr_regressor)
       print('learning rate is '+str(i)+' nestimators is '+str(j))
       print('R2 Score for RandomForestRegressor', xgbreg_r2_score)
       print('\n')

learning rate is 0.001 nestimators is 50
R2 Score for RandomForestRegressor 0.6885187129369406


learning rate is 0.001 nestimators is 100
R2 Score for RandomForestRegressor 0.6885187129369406


learning rate is 0.001 nestimators is 250
R2 Score for RandomForestRegressor 0.6885187129369406


learning rate is 0.001 nestimators is 500
R2 Score for RandomForestRegressor 0.6885187129369406


learning rate is 0.001 nestimators is 700
R2 Score for RandomForestRegressor 0.6885187129369406


learning rate is 0.001 nestimators is 750
R2 Score for RandomForestRegressor 0.6885187129369406


learning rate is 0.001 nestimators is 800
R2 Score for RandomForestRegressor 0.6885187129369406


learning rate is 0.001 nestimators is 850
R2 Score for RandomForestRegressor 0.6885187129369406


learning rate is 0.001 nestimators is 900
R2 Score for RandomForestRegressor 0.6885187129369406


learning rate is 0.001 nestimators is 950
R2 Score for RandomForestRegressor 0.6885187129369406


learning rate is 0.00

In [64]:
xgbr_regressor = XGBRegressor(learning_rate=0.1, n_estimators=100, n_jobs=-1)
xgbr_regressor.fit(X_train, y_train)


XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.1, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100,
             n_jobs=-1, num_parallel_tree=1, predictor='auto', random_state=0,
             reg_alpha=0, reg_lambda=1, ...)

In [23]:
"""
xgbr_regressor.feature_importances_
xgbr_regressor.feature_names_in_

fig = plt.figure(figsize=(20, 25))
plt.barh(xgbr_regressor.feature_names_in_, xgbr_regressor.feature_importances_)
plt.xlabel("Courses offered")
plt.ylabel("No. of students enrolled")
plt.title("Students enrolled in different courses")
plt.show()
"""

'\nxgbr_regressor.feature_importances_\nxgbr_regressor.feature_names_in_\n\nfig = plt.figure(figsize=(20, 25))\nplt.barh(xgbr_regressor.feature_names_in_, xgbr_regressor.feature_importances_)\nplt.xlabel("Courses offered")\nplt.ylabel("No. of students enrolled")\nplt.title("Students enrolled in different courses")\nplt.show()\n'

#### Testing

In [65]:
xgbr_regressor_prediction = xgbr_regressor.predict(X_test)
xgbr_regresso_r2_score = r2_score(y_test, xgbr_regressor_prediction)
print('R2 Score for XGBRegressor', xgbr_regresso_r2_score)

from sklearn.metrics import mean_absolute_error
print("MAE",mean_absolute_error(y_test, xgbr_regressor_prediction))

filename = 'xgbr_regressor2.sav'
pickle.dump(xgbr_regressor, open(filename, 'wb'))

R2 Score for XGBRegressor 0.43663564048300896
MAE 1030572.125


In [25]:
"""

from xgboost import plot_importance


fig = plt.figure(figsize=(115, 95))
plot_importance(xgbr_regressor,height=25.2)
plt.show()
"""
feature_important = xgbr_regressor.get_booster().get_score(importance_type='weight')
keys = list(feature_important.keys())
values = list(feature_important.values())

data = pd.DataFrame(data=values, index=keys, columns=["score"]).sort_values(by = "score", ascending=False)
data.nlargest(40, columns="score").plot(kind='barh', figsize = (30,20)) ## plot top 40 features


In [26]:
"""
data = pd.DataFrame(columns=[X.columns.values,'bonjour'])
data.loc['latitude']=1.2
"""

"\ndata = pd.DataFrame(columns=[X.columns.values,'bonjour'])\ndata.loc['latitude']=1.2\n"

In [27]:
data.head()

,score
latitude,840.0
size_in_sqft,622.0
longitude,588.0
no_of_bedrooms,203.0
no_of_bathrooms,70.0


In [28]:
#data.

In [29]:
"""
import gradio as gr

def greet(latitude=0,longitude=0,size_in_sqft=0,no_of_bedrooms=1,no_of_bathrooms=1, view_of_water=True, unfurnished=False,covered_parking=False,pets_allowed=False, kitchen_appliances=False, concierge=False):
    print('bonjour')
    tab =['latitude','longitude','size_in_sqft','no_of_bedrooms','no_of_bathrooms','covered_parking','unfurnished','concierge','kitchen_appliances','pets_allowed','view_of_water']
    print(latitude)
    print(longitude)
    print(unfurnished)
    dt = pd.DataFrame({'latitude': [latitude],
                        'longitude': [longitude],
                        'size_in_sqft':  [size_in_sqft],
                        'no_of_bedrooms':  [no_of_bedrooms],
                        'no_of_bathrooms':  [no_of_bathrooms],
                        'view_of_water':  [1] if view_of_water==True else [0],
                        'unfurnished':  [1] if unfurnished==True else [0],
                        'covered_parking':  [1] if covered_parking==True else [0],
                        'pets_allowed': [1] if pets_allowed==True else [0],
                        'kitchen_appliances': [1] if kitchen_appliances==True else [0],
                        'concierge': [1] if concierge==True else [0]
                        } )
    
         #print('X.columns',X.columns)
    
    dt.head        
    print(dt['latitude'])
    print(X.shape)
    
    
    filename = 'GradientBoostingRegressor2.sav'
    loaded_model = pickle.load(open(filename, 'rb'))
    result = loaded_model.predict(dt)
    result = result[0] *0.27
         
    print('result',result) 
    #dt.head()
    return result   
    #gradient_predictions = gradient.predict(tab)
    #return gradient_predictions

demo = gr.Interface(
    fn=greet,
    inputs=["number", "number", "number", "number","number","checkbox","checkbox","checkbox","checkbox","checkbox","checkbox"],
    outputs=["number"],
)
demo.launch()
"""

'\nimport gradio as gr\n\ndef greet(latitude=0,longitude=0,size_in_sqft=0,no_of_bedrooms=1,no_of_bathrooms=1, view_of_water=True, unfurnished=False,covered_parking=False,pets_allowed=False, kitchen_appliances=False, concierge=False):\n    print(\'bonjour\')\n    tab =[\'latitude\',\'longitude\',\'size_in_sqft\',\'no_of_bedrooms\',\'no_of_bathrooms\',\'covered_parking\',\'unfurnished\',\'concierge\',\'kitchen_appliances\',\'pets_allowed\',\'view_of_water\']\n    print(latitude)\n    print(longitude)\n    print(unfurnished)\n    dt = pd.DataFrame({\'latitude\': [latitude],\n                        \'longitude\': [longitude],\n                        \'size_in_sqft\':  [size_in_sqft],\n                        \'no_of_bedrooms\':  [no_of_bedrooms],\n                        \'no_of_bathrooms\':  [no_of_bathrooms],\n                        \'view_of_water\':  [1] if view_of_water==True else [0],\n                        \'unfurnished\':  [1] if unfurnished==True else [0],\n                  

In [30]:
#keras
from tensorflow import keras
from PIL import ImageFont
#from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.preprocessing.text import one_hot
from keras.layers import Dense, Activation, Flatten

import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 
from matplotlib import pyplot as plt
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [80]:
"""
NN_model = Sequential()

# The Input Layer :
NN_model.add(Dense(128, kernel_initializer='normal',input_dim = X.shape[1], activation='relu'))

# The Hidden Layers :
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
#NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
#NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))


# The Output Layer :
NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
NN_model.summary()
"""

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 128)               4352      
                                                                 
 dense_10 (Dense)            (None, 256)               33024     
                                                                 
 dense_11 (Dense)            (None, 1)                 257       
                                                                 
Total params: 37,633
Trainable params: 37,633
Non-trainable params: 0
_________________________________________________________________


In [81]:
"""
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]
"""

In [82]:
"""history = NN_model.fit(X, y, epochs=200, batch_size=32, validation_split = 0.2, callbacks=callbacks_list)"""

Epoch 1/200
43/51 [========================>.....] - ETA: 0s - loss: 2359495.7500 - mean_absolute_error: 2359495.7500
Epoch 1: val_loss improved from inf to 3241739.00000, saving model to Weights-001--3241739.00000.hdf5
51/51 [==============================] - 2s 17ms/step - loss: 2333475.2500 - mean_absolute_error: 2333475.2500 - val_loss: 3241739.0000 - val_mean_absolute_error: 3241739.0000
Epoch 2/200
49/51 [===========================>..] - ETA: 0s - loss: 2315450.7500 - mean_absolute_error: 2315450.7500
Epoch 2: val_loss improved from 3241739.00000 to 3209105.75000, saving model to Weights-002--3209105.75000.hdf5
51/51 [==============================] - 0s 9ms/step - loss: 2318887.0000 - mean_absolute_error: 2318887.0000 - val_loss: 3209105.7500 - val_mean_absolute_error: 3209105.7500
Epoch 3/200
47/51 [==========================>...] - ETA: 0s - loss: 2245814.2500 - mean_absolute_error: 2245814.2500
Epoch 3: val_loss improved from 3209105.75000 to 3098889.25000, saving model to W

In [83]:
"""import seaborn as sns
import matplotlib.pyplot as plt

history_df = pd.DataFrame.from_dict(history.history)
sns.lineplot(data=history_df[['mean_absolute_error', 'val_mean_absolute_error']])
plt.xlabel("epochs")
plt.ylabel("MAE")"""

Text(221.472,0.5,'MAE')